In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from flipside import Flipside
flipside = Flipside("023b83a2-e411-4849-a961-3742c87bc9c9", "https://api-v2.flipsidecrypto.xyz")

We first set the symbol of the token we want to query data from.

In [2]:
token = "sETH2"

This write the sql code and start query.

In [3]:
sql = """
SELECT HOUR, SYMBOL, PRICE
FROM ethereum.core.fact_hourly_token_prices
""" +\
"WHERE SYMBOL = '" + token + "'" 

query_result_set = flipside.query(sql, page_number = 1, page_size = 50000)

current_page_number = 1
page_size = 50000
total_pages = 2

# All the data will be stored in a list called "all_rows"
all_rows = []

while current_page_number <= total_pages:
  results = flipside.get_query_results(
    query_result_set.query_id,
    page_number=current_page_number,
    page_size=page_size
  )

  total_pages = results.page.totalPages
  if results.records:
      all_rows = all_rows + results.records
  
  current_page_number += 1

# Merge all rows to a dataframe
df = pd.DataFrame(all_rows)
df.drop(columns = ['__row_index'], inplace = True)
df['hour'] = pd.to_datetime(df['hour'])
df.sort_values(by = ['hour'], inplace = True)
df.head(10)

,hour,symbol,price
4654,2021-06-24 17:00:00+00:00,sETH2,1918.88
4653,2021-06-24 18:00:00+00:00,sETH2,1918.88
4651,2021-06-24 19:00:00+00:00,sETH2,1918.88
4656,2021-06-24 20:00:00+00:00,sETH2,1918.88
4657,2021-06-24 21:00:00+00:00,sETH2,1918.88
4655,2021-06-24 22:00:00+00:00,sETH2,1885.39
4652,2021-06-24 23:00:00+00:00,sETH2,1885.39
4679,2021-06-25 00:00:00+00:00,sETH2,1885.39
4677,2021-06-25 01:00:00+00:00,sETH2,1885.39
4672,2021-06-25 02:00:00+00:00,sETH2,1885.39


Lastly, we save the file.

In [4]:
df.to_csv("/Users/zhicong/Dropbox/DEXs/Data/token_prices/" + token + ".csv", index=False)